In [ ]:
#!/usr/bin/env python

In [ ]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [ ]:
import wuml
from wuml.IO import *
from precision_recall import *
import numpy as np
import torch
import wplotlib
import torch.nn as nn
 

If we include the 1/0 of preterm birth info, would it help? <br>
since we do a good job in classification, we can leverage that

In [ ]:
def costFunction(x, y, ŷ, ind):
	n = x.shape[0]
	relu = nn.ReLU()
	R = torch.sum((y - ŷ) ** 2)
	R3 = 60*torch.sum(relu((ŷ - 42)))	# if prediction above 43, its wrong
	R4 = 60*torch.sum(relu((23 - ŷ)))	# if prediction below 22, its wrong
	R5 = 160*(torch.sum(relu((ŷ - y)))*torch.sum(relu((37-y))))/(n*n)
	loss = R + R3 + R4 + R5
	return loss

In [ ]:
data = wuml.wData(xpath='./data/D3_Imputed_Balanced_regression.csv', batch_size=32, 
					label_type='continuous', label_column_name='gestationAge', first_row_is_label=True)

In [ ]:
[X_train, X_test, y_train, y_test] = wuml.split_training_test(data, test_percentage=0.1)

In [ ]:
y_train_pre = X_train.pop_column('preterm_best')
y_test_pre = X_test.pop_column('preterm_best')

In [ ]:
X_train.Data_preprocess()
X_test.Data_preprocess()

In [ ]:
X_train.append_columns(y_train_pre)
X_test.append_columns(y_test_pre)

Objective 1 --------------------------------------------<br>
un_list = [costFunction, costFunction2, costFunction3, costFunction4, costFunction5, costFunction6, costFunction7]

In [ ]:
fun_list = [costFunction]

In [ ]:
for foo in fun_list:
	bNet = wuml.basicNetwork(foo, X_train, networkStructure=[(600,'relu'),(600,'relu'),(600,'relu'),(1,'none')], max_epoch=4000, learning_rate=0.01)
	bNet.train(print_status=True)
	
	#	This is the objective network output Training error
	ẙ = bNet(X_train, output_type='ndarray')	
	gestational_precision_recall(ẙ, y_train_pre)
	res = wuml.output_regression_result(y_train, ẙ, print_out=['mean absolute error'])
	
	#	This is the objective network output Test error
	ẙ = bNet(X_test, output_type='ndarray')	
	gestational_precision_recall(ẙ, y_test_pre)
	res = wuml.output_regression_result(y_test, ẙ, print_out=['mean absolute error', 'true label vs prediction table'])